참고링크
- https://towardsdatascience.com/fine-tune-a-non-english-gpt-2-model-with-huggingface-9acc2dc7635b
- https://colab.research.google.com/github/philschmid/fine-tune-GPT-2/blob/master/Fine_tune_a_non_English_GPT_2_Model_with_Huggingface.ipynb#scrollTo=hKBSyNLgqF9K

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
pip install transformers

In [3]:
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead
from transformers import pipeline

import torch
import torch.nn as nn
import json
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import warnings; warnings.filterwarnings("ignore")

In [4]:
with open('cosmetic_comment_list_all.json', 'r', encoding='utf-8') as f:
    data1 = json.load(f, strict=False)
with open('final_other_platform0.15.json', 'r', encoding='utf-8') as f:
    data2 = json.load(f, strict=False)

In [5]:
txt1 = [data1[i]['review_text'] for i in range(len(data1))]
# txt2 = [data2[i]['cmt_body'] for i in range(len(data2))]
txt2 = [data2[i] for i in range(len(data2))]

In [6]:
txt = txt1 + txt2
random.shuffle(txt)

In [7]:
len(txt)

237554

In [8]:
test_size = 0.05

In [9]:
train, test = train_test_split(txt, test_size=test_size, shuffle=True)

In [10]:
print(len(train), len(test))

225676 11878


In [11]:
train_concat = ' '.join(train)
test_concat = ' '.join(test)
train_concat = train_concat.replace('\n', '  ')
test_concat = test_concat.replace('\n', '  ')

In [12]:
with open('train_dataset_v3.txt', 'w') as f:
    f.write(train_concat)
with open('test_dataset_v3.txt', 'w') as f:
    f.write(test_concat)

In [13]:
tokenizer = AutoTokenizer.from_pretrained("models/tokenizer")
# tokenizer.save_pretrained("./models/tokenizer/")

In [14]:
train_path = 'train_dataset_v3.txt'
test_path = 'test_dataset_v3.txt'

In [15]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

In [16]:
train_dataset, test_dataset,data_collator = load_dataset(train_path, test_path, tokenizer)

In [17]:
model = AutoModelWithLMHead.from_pretrained("skt/kogpt2-base-v2")

In [18]:
training_args = TrainingArguments(
    output_dir="./cosmetics_kogpt_v2", #The output directory

    overwrite_output_dir=True, #overwrite the content of the output directory
    
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=64, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps= 800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    # no_cuda=True
    )

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

#### Custom loss function 설정하기
- 참고: https://huggingface.co/docs/transformers/main_classes/trainer

In [ ]:
# class Trainer_with_custom_loss(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.get("labels")
#         outputs = model(**inputs)
#         logits = outputs.get('logits')
#         loss_fct = nn.BCEWithLogitsLoss()
#         loss = loss_fct(logits.view(-1, self.model.config.num_labels),
#                         labels.float().view(-1, self.model.config.num_labels))
#         return (loss, outputs) if return_outputs else loss

In [ ]:
# trainer = Trainer_with_custom_loss(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
# )

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 116441
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 9100


Step,Training Loss


In [ ]:
trainer.save_model()

### Generation 

In [ ]:
generator = pipeline('text-generation', model='./gpt2_cosmetic_review_3ep', tokenizer="skt/kogpt2-base-v2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def review_generate(keyword, generator):
    return print(generator(keyword)[0]['generated_text'])

In [ ]:
review_generate('블러셔', generator)

블러셔로 써도 예뻐요!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
